In [ ]:
import roboticstoolbox as rtb
from spatialmath import SE3
import numpy as np

"""
# We will load a model of the Franka-Emika Panda robot defined by a URDF file
robot = rtb.models.Panda()
print(robot)

# The symbol @ indicates the link as an end-effector, a leaf node in the rigid-body tree. We will compute the forward kinematics next
Te = robot.fkine(robot.qr)  # forward kinematics
print(Te)


# We can solve inverse kinematics very easily. We first choose an SE(3) pose defined in terms of position and 
# orientation (end-effector z-axis down (A=-Z) and finger orientation parallel to y-axis (O=+Y)).
Tep = SE3.Trans(0.6, -0.3, 0.1) * SE3.OA([0, 1, 0], [0, 0, -1])
sol = robot.ik_LM(Tep)         # solve IK
print(sol)

q_pickup = sol[0]
print(robot.fkine(q_pickup))    # FK shows that desired end-effector pose was achieved

# We can animate a path from the ready pose qr configuration to this pickup configuration
qt = rtb.jtraj(robot.qr, q_pickup, 50)
robot.plot(qt.q, backend='pyplot', movie='panda1.gif')
"""

# Definition of the Lego robot arm structure (links and joints). This is a right arm:
E1 = rtb.ET.tz(0.50)     # 50 cm up
E2 = rtb.ET.Ry()         # Shoulder rotation forward/back                 (positive angle is backwards)
E3 = rtb.ET.ty(-0.10)    # 10 cm out to next shoulder joint
E4 = rtb.ET.Rx()         # Shoulder rotation out/up / in/down to the side (positive angle is in/down)
E5 = rtb.ET.tz(-0.15)    # 15 cm down to overarm rotation joint
E6 = rtb.ET.Rz()         # Overarm rotation                               (positive angle is counter clockwise in towards the body)
E7 = rtb.ET.tz(-0.15)    # 15 cm down to elbow
E8 = rtb.ET.Ry()         # Elbow rotation                                 (positive angle is extending the elbow)
E9 = rtb.ET.tx(0.10)     # 10 cm out to underarm rotation joint
E10 = rtb.ET.Rx()        # Underarm rotation                              (positive angle is rotating the underarm clockwise)
E11 = rtb.ET.tx(0.15)    # 15 cm out to wrist
E12 = rtb.ET.Rz()        # Wrist rotation                                 (positive angle is rotating the wrist downwards/inwards)

robot_arm = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12

# View the ETS
print(robot_arm)
print()

# print the number of joints in the robot model
print(f"The robot has {robot_arm.n} joints")

# print the number of ETs in the robot model
print(f"The robot has {robot_arm.m} ETs")

# Using the above methodolgy, we can calculate the forward kinematics of our robot model
# First, we must define the joint coordinates q, to calculate the forward kinematics at
neutral_pose_with_bent_elbow_q = np.array([0, 0, 0, 0, 0, 0])

# The ETS class has the .fkine method which can calculate the forward kinematics
# The .fkine methods returns an SE3 object
print(f"\nneutral_pose_with_bent_elbow: \n{robot_arm.fkine(neutral_pose_with_bent_elbow_q)}")

elbow_extended_q = np.deg2rad(np.array([0, 0, 0, 90, 0, 0]))
print(f"elbow_extended: \n{robot_arm.fkine(elbow_extended_q)}")

# Choose an SE(3) pose defined in terms of position and orientation 
# (end-effector z-axis down (A=-Z) and finger orientation parallel to y-axis (O=+Y)).
wanted_pose = SE3.Trans(0.25, -0.1, 0.2) * SE3.OA([1, 1, 0], [0, 0, 1])
#wanted_pose = SE3.Trans(0.25, -0.1, 0.2) * SE3.OA([0, 1, 0], [0, 0, 1]) # Same as starting pose, to test sanity of IK solution and model.
joint_angles_solution = robot_arm.ik_LM(wanted_pose)         # solve IK
print(f"\nSolution to the inverse kinematics problem: \n{joint_angles_solution}")

q_wanted_pose = joint_angles_solution[0]
#q_wanted_pose = np.deg2rad(np.array([0, 0, 0, 0, 0, 90]))
print(f"Joint angles for the wanted pose in degrees: {np.round(np.rad2deg(q_wanted_pose))}")
print(robot_arm.fkine(q_wanted_pose))    # FK shows that desired end-effector pose was achieved

# We can animate a path from the ready pose (qr configuration) to the new wanted position and orientation
qt = rtb.jtraj(neutral_pose_with_bent_elbow_q, q_wanted_pose, 50)
robot_arm.plot(qt.q, backend='pyplot', movie='robot_arm_1.gif')
